In [1]:
import requests
import re
import urllib
from bs4 import BeautifulSoup

In [2]:
def __has_valid_parentheses(s):
    count = 0
    for i in s:
        if i == "(":
            count += 1
        elif i == ")":
            count -= 1
        if count < 0:
            return False
    return count == 0

def __check_bad_movie_type(s):
    bad_types = ['TV Series', 'Short', 'Video Game', 'Video short', 'Video', 'TV Movie', 'TV Mini-Series', 'TV Series short', 'TV Special', 'Documentary short', 'Documentary', 'voice', 'uncredited', 'unconfirmed']
    for bad_type in bad_types:
        if bad_type in s:
            return True
    return False

def __check_bad_release(el) -> bool:
    result = el.find_all('a')
    if len(result) != 1:
        return True
    return False

def __extract_year(el) -> None:
    result = el.find('span', attrs={'class': 'year_column'})
    if len(result) != 0:
        result.extract()

def __extract_movie_name_url(el) -> (str, str):
    result = el.find('b')
    result.extract()
    return result.text.strip(), result.find('a')['href'] 
    

In [6]:
_url = 'https://www.imdb.com/'
headers = {'Accept-Language': 'en', 'X-FORWARDED-FOR': '2.21.184.0'}

url = 'https://www.imdb.com/name/nm0272581/'

In [7]:
response = requests.get(url, headers=headers)
assert response.ok
soup = BeautifulSoup(response.text, "html.parser")

In [36]:
output = []
result = soup.find_all('div', attrs={'id': re.compile('actor-*'), 'class': 'filmo-row'})
# check if it is actor or actress
if len(result) == 0:
    result = soup.find_all('div', attrs={'id': re.compile('actress-*'), 'class': 'filmo-row'})
assert len(result) != 0, 'Something wrong with the actor/actress soup.'
for el in result:
    # get rid of year
    __extract_year(el)

    # check for the release and movie type
    if __check_bad_release(el) or __check_bad_movie_type(el.text.strip()):
        continue

    # get movie name and url
    movie_name, movie_url = __extract_movie_name_url(el)
    # get element text
    text = el.text.strip()

    # check if there are some unidentified brackets left
    if '(' in text and __has_valid_parentheses(text):
        print('='*25)
        print(f'{movie_name} has some unidentified brackets:\n{text}')
    
    output.append((movie_name, urllib.parse.urljoin(_url, movie_url)))

Ziegfeld Follies has some unidentified brackets:
Fred Astaire ('Here's to the Ladies') / Raffles ('This Heart of Mine') / Tai Long ('Limehouse Blues') / ...


In [8]:
soup.find('span', attrs={'itemprop'}).text

'Rebecca Ferguson'